In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import albumentations as A # 데이터 증식

In [2]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5), # 좌우 반전
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5), # 밝기 및 대비 조절
    A.ShiftScaleRotate(p=0.5), # 이동, 확대/축소, 회전
])

data = []
labels = []

# Clean 폴더 이미지 불러오기
clean_dir = 'Data_PlasticCup/clean'
for filename in os.listdir(clean_dir):
    img = cv2.imread(os.path.join(clean_dir, filename))
    img = cv2.resize(img, (128, 128))
    augmented = transform(image=img)
    augmented_img = augmented['image']
    data.append(augmented_img)
    labels.append(0)  # 0 = 깨끗한 컵

# NG 폴더 이미지 불러오기
ng_dir = 'Data_PlasticCup/ng'
for filename in os.listdir(ng_dir):
    img = cv2.imread(os.path.join(ng_dir, filename))
    img = cv2.resize(img, (128, 128))
    augmented = transform(image=img)
    augmented_img = augmented['image']
    data.append(augmented_img)
    labels.append(1)  # 1 = 더러운 컵

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [7]:
# 모델 정의
model = Sequential()

# 컨볼루션 레이어 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 2 
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 3
model.add(Conv2D(128, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))

# fully connected 레이어
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# 데이터를 NumPy 배열로 변환
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=30, batch_size=32)

# 모델 저장
model.save('plastic_cup_classifier.h5')

Epoch 1/30
10/10 [==============================] - 2s 111ms/step - loss: 23.1981 - accuracy: 0.6930 - val_loss: 0.1950 - val_accuracy: 0.9494
Epoch 2/30
10/10 [==============================] - 1s 94ms/step - loss: 0.2923 - accuracy: 0.8924 - val_loss: 0.2189 - val_accuracy: 0.9241
Epoch 3/30
10/10 [==============================] - 1s 90ms/step - loss: 0.1972 - accuracy: 0.9304 - val_loss: 0.2302 - val_accuracy: 0.9114
Epoch 4/30
10/10 [==============================] - 1s 89ms/step - loss: 0.1052 - accuracy: 0.9557 - val_loss: 0.3068 - val_accuracy: 0.8861
Epoch 5/30
10/10 [==============================] - 1s 89ms/step - loss: 0.0832 - accuracy: 0.9715 - val_loss: 0.2484 - val_accuracy: 0.9747
Epoch 6/30
10/10 [==============================] - 1s 89ms/step - loss: 0.0304 - accuracy: 0.9905 - val_loss: 0.2353 - val_accuracy: 0.9747
Epoch 7/30
10/10 [==============================] - 1s 88ms/step - loss: 0.0600 - accuracy: 0.9778 - val_loss: 0.3410 - val_accuracy: 0.9747
Epoch 8/30


c:\Users\lavie\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model('plastic_cup_classifier.h5')

# 테스트 데이터셋 준비
x_test = np.array(x_val)
y_test = np.array(y_val)

# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc * 100:.2f}%')

# 예측값 구하기
y_pred = (model.predict(x_test) > 0.5).astype(int)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (using sklearn.metrics): {accuracy * 100:.2f}%')

3/3 - 0s - loss: 1.0124 - accuracy: 0.9747 - 268ms/epoch - 89ms/step
Test accuracy: 97.47%
3/3 [==============================] - 0s 30ms/step
Accuracy (using sklearn.metrics): 97.47%
